<a href="https://colab.research.google.com/github/Aryan556gaur/Langchain_project/blob/main/Langchain_Faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is link to cleaned preprocessed data https://drive.google.com/drive/folders/1c2ONCSSgnRt8dIfBOXK4fTOHsVumj3vX?usp=sharing

In [ ]:
https://drive.google.com/drive/folders/1c2ONCSSgnRt8dIfBOXK4fTOHsVumj3vX?usp=sharing

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4
!pip install -qU langchain-google-genai
!pip install -q pdfminer.six
!pip install -q faiss-cpu
!pip install -q sentence-transformers
!pip install -q google-generativeai

In [ ]:
!pip install -q langchain openai tiktoken rapidocr-onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 50.2 MB/s eta 0:00:00


In [ ]:
import os
import re
import getpass
from textblob import TextBlob
from pdfminer.high_level import extract_text
from nltk.corpus import stopwords
import string
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
import google.generativeai as genai
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# Define function to remove punctuation
exclude = set(string.punctuation)
def remove_punctuation(text):
    return ''.join(ch for ch in text if ch not in exclude)

# Define function to remove stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    return ' '.join(word for word in text.split() if word.lower() not in stop_words)

def remove_citation_and_refernces(text):
    return re.sub(r'\[([^\]]*)\]', '', text)

# Path to the folder containing PDF files
pdf_folder = 'https://drive.google.com/drive/folders/1mfPteojPewXLWgXMthS15D7S-525b5CA?usp=drive_link' #@param {type:"string"}

# Path to the folder where text files will be saved
text_folder = '/content/drive/MyDrive/pdfs' #@param {type:"string"}

# Create the text folder if it doesn't exist
os.makedirs(text_folder, exist_ok=True)

# Create text files for each PDF
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_file_path = os.path.join(pdf_folder, filename)
        text = extract_text(pdf_file_path)

        # Preprocess the text
        text = remove_punctuation(text)
        text = remove_stopwords(text)
        text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
        text = remove_citation_and_refernces(text)

        # Correcting the text (optional)
        textBlob = TextBlob(text)
        text = str(textBlob.correct())

        # Save the preprocessed text to a new file
        text_filename = os.path.splitext(filename)[0] + '.txt'
        text_file_path = os.path.join(text_folder, text_filename)
        with open(text_file_path, 'w', encoding='utf-8') as text_file:
            text_file.write(text)

In [ ]:
l=[]
text_folder='/content/drive/MyDrive/pdfs'
for f in os.listdir(text_folder):
  file_path = os.path.join(text_folder, f)
  with open(file_path,'r') as obj:
    l.append(obj.read())

full_data='\n'.join(l)
with open('file.txt','w') as f:
  f.write(full_data)

In [ ]:
queries=['What are the variety of Multimodal and Multi-modular AI Approaches to Streamline Autism Diagnosis in Young Children',
         'What is Autism Spectrum Disorder, how it is caysed',
         'What is the cure of Autism Spectrum Disorder',
         'What are Stereotypical and maladaptive behaviors in Autism Spectrum, how are these detected and managed',
         'How relevant is eye contact and how it can be used to detect Autism',
         'How can cross country trials help in development of Machine learning based Multimodal solutions ',
         'How early infants cry can help in the early detection of Autism ',
         'What are various methods to detect  Atypical Pattern of Facial expression in Children',
         'What kind of facial expressions can be used to detect Autism Disorder in children',
         'What are methods to detect Autism from home videos',
         'What is Still-Face Paradigm in Early Screening for High-Risk Autism Spectrum Disorder',
         'What is West Syndrome?',
         'What is the utility of Behavior and interaction imaging at 9 months of age predict autism/intellectual disability in high-risk infants with West syndrome']

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2',)

In [ ]:
loader=TextLoader('file.txt')
document=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
text_chunks=text_splitter.split_documents(document)
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2',)
vectorstore=FAISS.from_documents(text_chunks, embeddings)

for query in queries:
  query = query
  docs = vectorstore.similarity_search(query,5)
  print(f'query: {query} \n Top 5 similar document: {docs}')

  model=genai.GenerativeModel('gemini-pro')
  response=model.generate_content(f'summarize each part: {[i for i in docs]}')
  print(response.text)

  response=model.generate_content(f'give summarized summary of whole: {[i for i in docs]}')
  print(response.text)

query: What are the variety of Multimodal and Multi-modular AI Approaches to Streamline Autism Diagnosis in Young Children 
 Top 5 similar document: [Document(page_content='highrisk siblings32–35 introspective studies home movies3637 reported she city motor typical repertoire infants ASD ASD early social signals previously possessed automatized computational procedures rousing eye cracking early stages–40 vocal pro ductions41 analysis caustics first utterances cry episodes none done interactive setting study proposed paradise shift assessment infant behavior dynamic assessment interactions previously achieved introspective approaches using home movies24 aim', metadata={'source': 'file.txt'}), Document(page_content='depositors instructions run classified provided httpsgithubcomqandeeltvideophenotyping autismplos standard approaches diagnosing cutis spectrum disorder ASD evaluate 20 100 behavior take several hours complete part contributed long wait times diagnosis subsequent delays acce

KeyboardInterrupt: 